In [ ]:
import pandas as pd
import regex as re
data = pd.read_csv("/Users/ishaan/Downloads/archive (1)/master_season1-35.tsv", sep = "\t")

In [27]:
data.shape

(349641, 9)

In [2]:
data.head()

,round,value,daily_double,category,comments,answer,question,air_date,notes
0,1,100,no,LAKES & RIVERS,-,River mentioned most often in the Bible,the Jordan,1984-09-10,-
1,1,200,no,LAKES & RIVERS,-,Scottish word for lake,loch,1984-09-10,-
2,1,400,no,LAKES & RIVERS,-,American river only 33 miles shorter than the ...,the Missouri,1984-09-10,-
3,1,500,no,LAKES & RIVERS,-,"World's largest lake, nearly 5 times as big as...",the Caspian Sea,1984-09-10,-
4,1,100,no,INVENTIONS,-,Marconi's wonderful wireless,the radio,1984-09-10,-


In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
#!pip install sentence-transformers
#!pip install tf-keras
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

2025-04-22 13:31:01.595474: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
data.columns

Index(['round', 'value', 'daily_double', 'category', 'comments', 'answer',
       'question', 'air_date', 'notes'],
      dtype='object')

In [8]:
q_a = data[["answer", "question", "category"]]
q_a.head()

,answer,question,category
0,River mentioned most often in the Bible,the Jordan,LAKES & RIVERS
1,Scottish word for lake,loch,LAKES & RIVERS
2,American river only 33 miles shorter than the ...,the Missouri,LAKES & RIVERS
3,"World's largest lake, nearly 5 times as big as...",the Caspian Sea,LAKES & RIVERS
4,Marconi's wonderful wireless,the radio,INVENTIONS


In [9]:
def clean_text(text):
    cleaned_text =  re.sub(r'[^a-zA-Z0-9\s]', '', text.lower().strip())
    return cleaned_text

In [10]:
q_a['question'] = q_a['question'].astype(str).apply(clean_text)
q_a['answer'] = q_a['answer'].astype(str).apply(clean_text)
q_a['category'] = q_a['category'].astype(str).apply(clean_text)

In [11]:
q_a.head()

,answer,question,category
0,river mentioned most often in the bible,the jordan,lakes rivers
1,scottish word for lake,loch,lakes rivers
2,american river only 33 miles shorter than the ...,the missouri,lakes rivers
3,worlds largest lake nearly 5 times as big as s...,the caspian sea,lakes rivers
4,marconis wonderful wireless,the radio,inventions


In [13]:
#!pip install ipywidgets --upgrade
#!jupyter labextension install @jupyter-widgets/jupyterlab-manager@v2
#!jupyter lab build

In [18]:
#from tqdm import tqdm
#from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

# Apply tqdm to your pandas series
#q_a['q_embedding'] = q_a['question'].apply(lambda x: model.encode(x, convert_to_numpy=True))
#q_a['a_embedding'] = q_a['answer'].apply(lambda x: model.encode(x, convert_to_numpy=True))

In [11]:
#q_a.to_pickle("jeopardy_with_embeddings.pkl")

In [12]:
embeddings = pd.read_pickle("/Users/ishaan/Downloads/jeopardy_with_embeddings.pkl")
embeddings.head()

,answer,question,category,q_embedding,a_embedding
0,river mentioned most often in the bible,the jordan,lakes rivers,"[-0.00089073076, 0.12694257, -0.06492324, 0.04...","[0.04561206, 0.10563557, 0.0056359367, 0.00589..."
1,scottish word for lake,loch,lakes rivers,"[0.026929816, -0.010473593, 0.009912812, 0.039...","[0.0060076676, -0.008746384, -0.018628595, -0...."
2,american river only 33 miles shorter than the ...,the missouri,lakes rivers,"[0.017947417, 0.032795604, 0.010262107, 0.0986...","[0.09347568, 0.052192014, 0.002176766, -0.0128..."
3,worlds largest lake nearly 5 times as big as s...,the caspian sea,lakes rivers,"[-0.043156452, 0.09111538, -0.039893687, 0.019...","[0.05761624, 0.040556036, 0.024782812, -0.0531..."
4,marconis wonderful wireless,the radio,inventions,"[-0.011442868, 0.09393178, -0.092939794, 0.038...","[-0.1215708, 0.058970764, -0.00368905, -0.0221..."


## Spell Checking

In [13]:
#pip install symspellpy

from symspellpy import SymSpell, Verbosity

# Initialize
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

# Load dictionary
dictionary_path = "/Users/ishaan/Downloads/frequency_dictionary_en_82_765.txt"
term_index = 0       # column of the word
count_index = 1      # column of the frequency
sym_spell.load_dictionary(dictionary_path, term_index, count_index)


True

In [15]:
def correct_spelling_symspell(text):
    suggestions = sym_spell.lookup_compound(text, max_edit_distance=2)
    if suggestions:
        return suggestions[0].term
    return text


In [28]:
from sklearn.metrics.pairwise import cosine_similarity

def search(query, top_k=5):
    corrected_query = correct_spelling_symspell(query)
    print(f"Original: {query} | Corrected: {corrected_query}")
    
    query_vec = model.encode(corrected_query, convert_to_numpy=True).reshape(1, -1)
    matrix = np.vstack(embeddings['q_embedding'].values)
    scores = cosine_similarity(query_vec, matrix).flatten()
    
    top_indices = scores.argsort()[-top_k:][::-1]
    results = embeddings.iloc[top_indices][['question', 'answer', 'category']].copy()
    results['score'] = scores[top_indices]
    return results



In [33]:
query = "nanana"
results = search(query, top_k=5)
print(results)

Original: nanana | Corrected: banana
       question                                             answer  \
109836   banana  fruity first name of acclaimed japanese author...   
172050   banana                    harry belafonte  boat song dayo   
89084    banana  ecuador is the worlds largest exporter of this...   
146336   banana  a chiquita cocktail combines cointreau cream  ...   
50210    banana  ben  jerrys chunky monkey ice cream is this fr...   

                  category  score  
109836  20th century women    1.0  
172050   the musical fruit    1.0  
89084                  aaa    1.0  
146336     potent potables    1.0  
50210               sweets    1.0  
